## Menggunakan Decision Tree untuk memprediksi performa pelajar menggunakan data-data pelajar
Tujuan dari project ini adalah untuk memprediksi apakah pelajar akan lulus atau tidak berdasarkan data-data yang sudah dikumpulkan 

Link Dataset : https://archive.ics.uci.edu/ml/datasets/student+performance

In [ ]:
# import / load dataset yang akan digunakan dengan nama file student-por.csv
import pandas as pd
from google.colab import files
uploaded = files.upload()
d = pd.read_csv('student-por.csv', sep=';')

In [ ]:
# dari dataset yang di import berisikan total 649 data siswa
len(d)

# Pada dataset tersebut terdapat 3 nilai yaitu G1, G2, dan G3 dimana,

G1 --> nilai pertama

G2 --> nilai kedua

G3 --> nilai akhir

Rentang nilai G1 adalah 0-20, demikian juga dengan G2 dan G3

Indikator siswa lulus atau tidak lulus adalah dengan menjumlahkan nilai G1-G3, apabila nilai dari penjumlahan tersebut adalah lebih besar sama dengan 35 maka siswa tersebut dianggap lulus dan sebaliknya


Indikator lulus atau tidak lulus menggunakan biner atau boolean dimana,

1 (true) = lulus

0 (false) = tidak lulus

In [ ]:
# menambahkan kolom baru yaitu 'pass' berdasarkan perhitungan penjumlahan G1-G3
d['pass'] = d.apply(lambda row: 1 if (row['G1']+row['G2']+row['G3']) >= 35 else 0, axis=1)
d = d.drop(['G1', 'G2', 'G3'], axis=1)
d.head() # --> menampilkan 5 baris pertama dari dataset yang digunakan

## Menggunakan metode one-hot encoding untuk kolom-kolom yang menggunakan tipe biner/boolean. Lalu dikonversikan dan ditampilkan dalam bentuk biner.

In [ ]:
# menggunakan one-hot encoding untuk kolom kategorikal (kolom yang menggunakan biner/bool)
# konversikan ke biner 
d = pd.get_dummies(d, columns = ['sex', 'school', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob',
                                 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities',
                                 'nursery', 'higher', 'internet', 'romantic'])
d.head()

In [ ]:
# acak kolomnya
d = d.sample(frac=1)
# memisahkan data untuk training dan data untuk test
d_train = d[:500]
d_test = d[500:]

d_train_att = d_train.drop(['pass'], axis=1)
d_train_pass = d_train['pass']

d_test_att = d_test.drop(['pass'], axis=1)
d_test_pass = d_test['pass']

d_att = d.drop(['pass'], axis=1)
d_pass = d['pass']

# dihitung banyaknya siswa pasif dalam dataset kita
import numpy as np
print("Passing %d out of %d (%.2f%%)" % (np.sum(d_pass), len(d_pass), 100*float(np.sum(d_pass)) / len(d_pass)))

## Membuat model decision treenya

In [ ]:
# memasukkan ke model ke dalam decision tree
from sklearn import tree
t = tree.DecisionTreeClassifier(criterion="entropy", max_depth=5)
t = t.fit(d_train_att, d_train_pass)

In [ ]:
# simpan modelnya
tree.export_graphviz(t, out_file="student-performance.dot", label="all", impurity=False, proportion=True,
                     feature_names=list(d_train_att), class_names=["fall", "pass"],
                     filled=True, rounded=True)

In [ ]:
# perhitungan t.score berdasarkan d_test_att dan d_test_pass
t.score(d_test_att, d_test_pass)

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(t, d_att, d_pass, cv=5)
# menampilkan rata-rata skor siswa dan dua standar deviasinya 
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## Melakukan cross validation pada seluruh dataset yang akan membagi data pada salah satu dari 20/80, di mana 20% adalah testing set dan 80% pada training set. Rata-rata hasilnya 67%. Ini menunjukkan bahwa kami memiliki kumpulan data yang seimbang. Di sini kami memiliki berbagai pilihan yang harus diambil terkait max_depth:

In [ ]:
for max_depth in range(1, 20):
  t = tree.DecisionTreeClassifier(criterion="entropy", max_depth=max_depth)
  scores = cross_val_score(t, d_att, d_pass, cv=5)
  print ("Max depth: %d, Accuracy: %0.2f (+/- %0.2f)" % (max_depth, scores.mean(), scores.std() * 2))

In [ ]:
depth_acc = np.empty((19,3), float)
i = 0
for max_depth in range(1,20):
  t = tree.DecisionTreeClassifier(criterion="entropy", max_depth=max_depth)
  scores = cross_val_score(t, d_att, d_pass, cv=5)
  depth_acc[i,0] = max_depth
  depth_acc[i,1] = scores.mean()
  depth_acc[i,2] = scores.std() * 2
  i += 1

depth_acc

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.errorbar(depth_acc[:,0], depth_acc[:,1], yerr=depth_acc[:,2])
plt.show()